# Setup

## Imports

In [1]:
# General
import numpy as np
import pandas as pd

# Preprocessing
from gensim.models import FastText

In [2]:
import os
import sys
# Add the path to the utils folder
sys.path.append(os.path.abspath('../..'))
import importlib
# Custom modules
from utils import memory_usage
from config import run_config, MODELS_PATH, FEATURES_PATH, PROCESSED_DATA_PATH
importlib.reload(sys.modules['utils'])
importlib.reload(sys.modules['config'])

<module 'config' from 'e:\\College\\4- Senior 2\\Semester 1\\NLP\\Project\\config.py'>

## Config

In [3]:
run_config()

In [4]:
# from IPython.utils import io

# with io.capture_output() as captured:
#     %run ../../1-EDA_and_Preprocessing/Preprocessing/Preprocessing.ipynb

In [5]:
# df_train = pd.read_parquet(PROCESSED_DATA_PATH + '/train_preprocessing.parquet')
X_train = np.load(PROCESSED_DATA_PATH + '/X_train.npy', allow_pickle=True)
X_train[:5]

array([list(['three', 'pizzas', 'no', 'american', 'cheese', 'and', 'a', 'water', 'and', 'one', 'ginger', 'ale', 'and', 'a', 'san', 'pellegrino', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']),
       list(['three', 'large', 'pizzas', 'with', 'balsamic', 'glaze', 'and', 'three', 'party', 'sized', 'pies', 'with', 'just', 'a', 'little', 'cherry', 'tomato', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']),
       list(["i'd", 'like', 'a', 'lunch', 'sized', 'pizza', 'with', 'parsley', 'fried', 'onions', 'and', 'parmesan', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']),
       list(['two', 'regular', 'pizzas', 'without', 'any', 'caramelized', 'onions', '<PAD>', '<PAD>', '<PAD>',

# FastText

[Add padding & <UNK> to word2vec model](https://gist.github.com/timotta/ceaec141cbb0b18740b5c48071d989ae)

In [ ]:
model_name = "/fast_text_model.bin"
update_model = False
if update_model or not os.path.exists(MODELS_PATH + model_name):
    print(f"Creating '{model_name}'...")
    # Create a FastText model
    EMBED_SIZE = 1024
    fast_text_model = FastText(sentences=X_train, vector_size=EMBED_SIZE, window=5, min_count=1, workers=4)
    fast_text_model.wv.add_vector("<UNK>", np.zeros(EMBED_SIZE))
    fast_text_model.wv["<PAD>"] = np.zeros(EMBED_SIZE)

    # Save the trained model
    print(f"Saving '{model_name}'...")
    fast_text_model.save(MODELS_PATH + model_name)
else:
    print(f"Loading '{model_name}'...")
    # Load the trained model
    fast_text_model = FastText.load(MODELS_PATH + model_name)
    
fast_text_model

Creating '/fast_text_model.bin'...


In [20]:
def load_fasttext_embeddings(model_name: str):
    print(f"Loading '{model_name}'...")
    return FastText.load(MODELS_PATH + model_name)

def get_word_embedding(fast_text_model, word):
    try:
        return fast_text_model.wv[word]
    except:
        return fast_text_model.wv["<UNK>"]
    
def get_sentence_embedding(fast_text_model, sentence):
    return np.array([get_word_embedding(fast_text_model, word) for word in sentence])

In [27]:
word2vec_model = load_fasttext_embeddings("/fast_text_model.bin")
get_sentence_embedding(fast_text_model, X_train[0])

Loading '/word2vec_model.bin'...


array([[-0.7180512 ,  0.00584892,  0.08585259, ..., -1.3326504 ,
        -0.3574455 ,  1.9497931 ],
       [-0.3477764 , -1.7241757 ,  0.16368103, ..., -0.32398897,
        -0.5057439 ,  0.29677933],
       [-0.25654992, -0.07621569, -1.0797989 , ..., -0.26241708,
         1.1733955 , -0.43284574],
       ...,
       [ 0.05875031,  0.70974684,  0.297136  , ..., -0.31129286,
        -0.2553544 ,  0.5461506 ],
       [ 0.05875031,  0.70974684,  0.297136  , ..., -0.31129286,
        -0.2553544 ,  0.5461506 ],
       [ 0.05875031,  0.70974684,  0.297136  , ..., -0.31129286,
        -0.2553544 ,  0.5461506 ]], dtype=float32)

In [ ]:
# X_train_embeddings = np.array([get_sentence_embedding(word2vec_model, sentence) for sentence in X_train])

In [3]:
memory_usage()

118.796875